In [ ]:





# Run this to install the appropriate tensorflow package
!pip install tensorflow==2.2.0


     |████████████████████████████████| 516.2MB 35kB/s 
     |████████████████████████████████| 3.0MB 11.4MB/s 
     |████████████████████████████████| 460kB 42.8MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


**The above code section is from the template provided by the AI Challenge organiser.**

**The following code section is from the template provided by the AI Challenge organiser.**

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, accuracy_score, classification_report
from tensorflow.keras import models, layers, optimizers
from tensorflow.python.keras.saving import hdf5_format
from keras.preprocessing.image import ImageDataGenerator, DirectoryIterator
import h5py, itertools, collections
import itertools

##################
# Verifications:
#################
print('GPU is used.' if len(tf.config.list_physical_devices('GPU')) > 0 else 'GPU is NOT used.')
print("Tensorflow version: " + tf.__version__)


GPU is used.
Tensorflow version: 2.5.0


Now, please run the following cell to download the dataset that you will use to build your model.

**The following code section is from the template provided by the AI Challenge organiser.**

In [ ]:
# Download dataset:
!wget http://206.12.93.90:8080/simpson_dataset/simpsons_train.tar.gz 
# Unzip the dataset:
!tar -xzvf simpsons_train.tar.gz > /dev/null


--2021-05-15 23:24:54--  http://206.12.93.90:8080/simpson_dataset/simpsons_train.tar.gz
Connecting to 206.12.93.90:8080... connected.
HTTP request sent, awaiting response... 200 
Length: 488194922 (466M) [application/x-gzip]
Saving to: ‘simpsons_train.tar.gz’

simpsons_train.tar. 100%[===================>] 465.58M   148MB/s    in 3.3s    

2021-05-15 23:24:57 (142 MB/s) - ‘simpsons_train.tar.gz’ saved [488194922/488194922]



**The following code section is from the template provided by the AI Challenge organiser.**

**Changes were made to:**

**- image size**

**- batch size**

**An imageDataGenerator was added to feed augmented images into the model for data augmentation.**

*   List item
*   List item



In [ ]:
'''
    Split train and validation.
'''
# We define the size of input images to 128x128 pixels.
image_size = (224, 224)
# We define the batch size
batch_size = 128

# Create an image generator with a fraction of images reserved for validation:
image_generator = ImageDataGenerator(validation_split=0.1)

# Now, we create a training data iterator by creating batchs of images of the same size as 
# defined previously, i.e., each image is resized in a 64x64 pixels format.
train_ds =  DirectoryIterator(
    "dataset/simpsons_train/",
    image_generator,
    class_mode='categorical',
    seed=1337,
    target_size=image_size,
    batch_size=batch_size,
    subset = 'training',
)

# Similarly, we create a validation data iterator by creating batchs of images of the same size as 
# defined previously, i.e., each image is resized in a 64x64 pixels format.
val_ds = DirectoryIterator(
    "dataset/simpsons_train/",
    image_generator,
    class_mode='categorical',
    seed=1337,
    target_size=image_size,
    batch_size=batch_size,
    subset = 'validation',
    shuffle=False
)

# We save the list of classes (labels).
class_names = list(train_ds.class_indices.keys())

# We also save the number of labels.
num_classes = train_ds.num_classes


Found 17603 images belonging to 20 classes.
Found 1945 images belonging to 20 classes.


# Exploring the data

Now, we do data exploration to show you samples of the images and their labels and some statistics to help you in understanding the data.

# Designing the model


We now design the architecture for the task. The artchitecture below consists of:
1. `Rescaling layer:` whose role is to normalize the input data to values between 0 and 1. This will help in speed up the training process.
2. `Flatten layer:` whose role is to flatten the 3D volume.
3. `Dense layers`: one dense layer followed by a classification layer with a softmax activation function.

Please note that you will have to design your own model if you want to beat the baseline and be at the top of the leaderboard!

In [ ]:
# CONVOLUTIONAL PRETRAINED BASE

conv_base = tf.keras.applications.DenseNet201(weights='imagenet', include_top=False)


74842112/74836368 [==============================] - 1s 0us/step


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
## LOAD PREVIOUS MODELS READY TO COMBINE

In [ ]:
# Reload Model 1

model_load_name = "ddnsnetAllAug_bch_0.99383a"
filepath = F"/content/gdrive/My Drive/{model_load_name}"
model_reload1 = keras.models.load_model(filepath)


In [ ]:
results = model_reload1.evaluate(val_ds)

In [ ]:
# Reload Model 2

model_load_name = "dnsnetaugSubt_0.99280"
filepath = F"/content/gdrive/My Drive/{model_load_name}"
model_reload2 = keras.models.load_model(filepath)


In [ ]:
results = model_reload2.evaluate(val_ds)

In [ ]:
model_reload1.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
densenet201 (Model)          multiple                  18321984  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              1967104   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                20500     
_________________________________________________________________
activation (Activation)      (None, 20)                0     

In [ ]:
ens_1_model = model_reload1


ens_2_model = model_reload2

#ens_6_model = model_reload6
#models_lst=[ens_1_model,ens_1b_model,ens_1c_model,ens_1d_model,ens_1e_model,ens_1f_model,ens_2_model,ens_3_model] 
#amodels_lst=[ens_1_model,ens_1b_model,ens_1d_model,ens_1e_model] 
#bmodels_lst=[ens_1_model,ens_1b_model,ens_2_model,ens_3_model] 

models_lst=[ens_1_model,ens_2_model] 

#dmodels_lst=[ens_1_model,ens_1b_model,ens_3_model]

In [ ]:
ens_1_model.summary()
ens_2_model.summary()




In [ ]:
#Remove softmax activation layer
def remove_softmax(m_list):
  new_model_lst=[]
  for model in m_list:
    new_mod=keras.Sequential()
    for layer in model.layers[:-1]:
      new_mod.add(layer)
    new_model_lst.append(new_mod)
  return new_model_lst

models_lst = remove_softmax(models_lst)


In [ ]:
from keras.models import Input
from keras import Model


def Create_ensemble(models, model_input):
    # list combining model outputs
    xx=[model(model_input) for model in models] 
    # averaging the model outputs
    avragelyr=layers.average(xx) 
    # add softmax
    sftmx=layers.Activation(activation='softmax')(avragelyr)
    # build model from same input and avg output
    modelEns = Model(inputs=model_input, outputs=sftmx,    name='ensemble')  
   
    return modelEns
#models_lst = [ens_1_model,ens_2_model,ens_3_model,ens_4_model,ens_5_model]
ens_input = Input(shape=ens_1_model.input_shape[1:])
new_ens_model = Create_ensemble(models_lst, ens_input)
new_ens_model.summary()


Model: "ensemble"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 20)           20309588    input_2[0][0]                    
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 20)           20309588    input_2[0][0]                    
__________________________________________________________________________________________________
average (Average)               (None, 20)           0           sequential[1][0]                 
                                                                 sequential_1[1][0]        

In [ ]:
new_ens_model.compile(optimizer=optimizers.RMSprop(lr=0.00001),
              loss='CategoricalCrossentropy',
              metrics=['accuracy'])

In [ ]:
results = new_ens_model.evaluate(val_ds)


In [ ]:
print(results[-1])
valdn_acc = results[-1]

In [ ]:
with h5py.File('/content/gdrive/My Drive/'+"EnsModel_1band3_"+"{:.7f}".format(valdn_acc)+ '_model.h5', mode='w') as f:
    hdf5_format.save_model_to_hdf5(new_ens_model, f)
    f.attrs['class_names'] = class_names
    f.attrs['image_size'] = image_size